<a href="https://colab.research.google.com/github/jadinsstewart/Congressional-Analytics/blob/main/Project_1_Part_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center> Analysis Comparing Political Affiliation by Number of Computing Devices Owned and Median Household Age </center></h1>

Ella Archie (earchie@calpoly.edu)

Jadin Stewart (jsstewar@calpoly.edu)

February 5th, 2023

# Introduction

Last week, we presented our analysis comparing district political affiliation and the percentage of households owning at least one computer in that district. While we found evidence to support our hypothesis that districts with a high percentage of households having 1+ computers tend to be democratic, we were left with a few questions we wanted to further investigate.

To better understand the impact that technological devices have on political affiliation, we have decided to investigate how the percentage of households having 1+ computing device (not limited to just a laptop or desktop) impacts a districts political affiliation. To take the investigation even further, we will investigate how median household age impacts how many households own at least one computing device.

# About

This document was created for our Cloud Computing Class at California Polytechnic State University. We gathered data for this analysis from several places:

We gathered census data which included total households per district and computing devices owned per household from the United States Census Bureau. (https://api.census.gov/data/2021/acs/acs1?get=NAME,B19001_001E,B28001_011E,B28001_003E&for=congressional%20district:*&in=state:*)

We gathered political affiliation data from 

# Hypothesis

# ACS Data

# Political Affiliation Data

# Merging the Data

# Visualization

# Findings